In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, get_close_matches
from tqdm import tqdm
import re

In [76]:
df_man = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v2/subsequent_manufacturers.csv')
# df_manufacturer = df_man.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
# df_manufacturer = df_manufacturer[df_manufacturer['manufacturer_match_score'] < 0.8]
# df_manufacturer.head()
df_man = df_man.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
df_man = df_man.drop_duplicates(subset=['manufacturer_name'], keep='first')
len(df_man)
len(df_man)

20051

In [3]:
df_non_dup = df_manufacturer.drop_duplicates(subset='manufacturer_name', keep='first')
df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')

manufacturer_list = []

for index, row in df_non_dup.iterrows():
    word_count = row['word_count']
    manufacturer_name = row['manufacturer_name']
    
    if word_count in [1,2]:
        manufacturer_slice = manufacturer_name.strip().split()[:1]
        manufacturer_list.append(' '.join(manufacturer_slice))
    elif word_count in [3,4,5]:
        manufacturer_slice = manufacturer_name.strip().split()[:2]
        manufacturer_list.append(' '.join(manufacturer_slice))
    else:
        manufacturer_slice = manufacturer_name.strip().split()[:3]
        manufacturer_list.append(' '.join(manufacturer_slice))
           
df_non_dup['manufacturer_slice'] = manufacturer_list        
df_non_dup = df_non_dup.reset_index(drop=True)
df_non_dup.head()

/tmp/ipykernel_5857/1627982089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
/tmp/ipykernel_5857/1627982089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')
/tmp/ipykernel_5857/1627982089.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice
0,local,oic ltd,0.15,1,local
1,fugo,pz cussons,0.27,1,fugo
2,mukurwe-ini dairy,zagro-india,0.41,2,mukurwe-ini
3,xxxx,apex,0.22,1,xxxx
4,g&g,hebei yuangzheng,0.20,1,g&g


In [4]:
df_non_dup.shape

(18619, 5)

In [5]:
df_manufacturer_slice = df_non_dup['manufacturer_slice'].drop_duplicates(keep='first').to_frame()
df_manufacturer_slice = df_manufacturer_slice.reset_index(drop=True)
df_manufacturer_slice

,manufacturer_slice
0,local
1,fugo
2,mukurwe-ini
3,xxxx
4,g&g
...,...
12693,gloria
12694,kiresi seed
12695,triachem tanzania
12696,tags


In [6]:
matched = []
def compare(i):
    compare = {}
    if i in matched:
        compare.update({i: ''})
    else:
        compare.update({i: get_close_matches(i, df_manufacturer_slice['manufacturer_slice'].to_list(), 20, 0.85)})
    # matched.extend(compare.values())
    matched.extend([item for sublist in compare.values() for item in sublist])
    manufacturer_slice = list(compare.keys())
    match = []
    for key, items in compare.items():
      match.append(items)
    return pd.Series([manufacturer_slice, match],index=['manufacturer_slice', 'match'])

      
cleaned_manufacturers_df = pd.DataFrame()
cleaned_manufacturers_df[['manufacturer_slice', 'match']] = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))
cleaned_manufacturers_df = cleaned_manufacturers_df.applymap(lambda x: x[0] if x else '')
cleaned_manufacturers_df.head()

,manufacturer_slice,match
0,local,"[local, localy, locals, locali, local., local,..."
1,fugo,"[fugo, fungo, fuo, fug]"
2,mukurwe-ini,[mukurwe-ini]
3,xxxx,"[xxxx, xxx]"
4,g&g,[g&g]


In [7]:
cleaned_manufacturers_df[150:200]

,manufacturer_slice,match
150,krickwood,[krickwood]
151,xxx,
152,lcal,
153,uganda house,[uganda house]
154,mamujee,[mamujee]
155,kzh,[kzh]
156,sayari crop,[sayari crop]
157,east africa,"[east africa, east. africa, east african, east..."
158,bright,"[bright, brighter]"
159,falcon/elgon/yara,[falcon/elgon/yara]


In [8]:
len(set(matched))

12698

In [9]:
dnd_cleaned_manufacturers_df = cleaned_manufacturers_df.copy()
cleaned_manufacturers_df['manufacturer_slice'] = cleaned_manufacturers_df['manufacturer_slice'].astype('str')

In [10]:
filtered_df = cleaned_manufacturers_df[cleaned_manufacturers_df['match'].apply(lambda x: len(x) == 0)]
filtered_df

,manufacturer_slice,match
84,somo,
131,intercheme,
151,xxx,
152,lcal,
192,loal,
...,...,...
12682,nexigen,
12684,amamy green,
12686,olympics,
12687,pt mandom,


In [11]:
for i, row in cleaned_manufacturers_df.iterrows():
    string = row['manufacturer_slice']
    lst = row['match']
    
    # Check if the string exists in any list of previous rows
    if not lst:
        for prev_i in range(i):
            prev_lst = cleaned_manufacturers_df.at[prev_i, 'match']
            if string in prev_lst:
                cleaned_manufacturers_df.at[i, 'match'] = prev_lst
                break  # Stop searching after finding the first match

# Reset the index of the DataFrame
# cleaned_manufacturers_df_subset = cleaned_manufacturers_df_subset.reset_index(drop=True)

pd.set_option('display.max_colwidth', None)
cleaned_manufacturers_df.head()

,manufacturer_slice,match
0,local,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]"
1,fugo,"[fugo, fungo, fuo, fug]"
2,mukurwe-ini,[mukurwe-ini]
3,xxxx,"[xxxx, xxx]"
4,g&g,[g&g]


In [12]:
dnd_df_non_dup = df_non_dup.copy()

In [13]:
df_non_dup = pd.merge(df_non_dup, cleaned_manufacturers_df, how='left', on='manufacturer_slice')
df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match
0,local,oic ltd,0.15,1,local,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]"
1,fugo,pz cussons,0.27,1,fugo,"[fugo, fungo, fuo, fug]"
2,mukurwe-ini dairy,zagro-india,0.41,2,mukurwe-ini,[mukurwe-ini]
3,xxxx,apex,0.22,1,xxxx,"[xxxx, xxx]"
4,g&g,hebei yuangzheng,0.20,1,g&g,[g&g]


In [14]:
# Initialize an empty list to store the similar strings
similar_strings = []

# Iterate over the rows of the DataFrame
for i, row in df_non_dup.iterrows():
    string = row['manufacturer_name']
    value = row['match']
    
    # Check if any other row has a similar value in 'col2'
    similar_rows = df_non_dup[df_non_dup['match'].apply(lambda x: x == value)]
    
    # Extract the strings from 'col1' in similar rows
    similar_strings.append(similar_rows['manufacturer_name'].tolist())

similar_strings[:5]

[['local',
  'lcal',
  'loal',
  'locl',
  'local,',
  'localy',
  'local manufacture',
  'local supplier',
  'local nairobi',
  'local manufacturer',
  'local artisan',
  'local juakali',
  'loca',
  'locali',
  'local providers',
  'local seller',
  'localy made',
  'local.',
  'local industries',
  'local manufactur',
  'local any',
  'local carpenters',
  'locals',
  'local ltd.'],
 ['fugo',
  'fug',
  'fugo company',
  'fuo',
  'fugo feeds',
  'fugo ltd',
  'fungo',
  'fugo vits',
  'fugo unga',
  'fugo feed',
  'fugo vit'],
 ['mukurwe-ini dairy'],
 ['xxxx', 'xxx'],
 ['g&g']]

In [15]:
df_similar_strings = pd.DataFrame({'similar_strings': similar_strings})
df_similar_strings = df_similar_strings['similar_strings'].drop_duplicates(keep='first').to_frame().reset_index(drop=True)
df_similar_strings

,similar_strings
0,"[local, lcal, loal, locl, local,, localy, local manufacture, local supplier, local nairobi, local manufacturer, local artisan, local juakali, loca, locali, local providers, local seller, localy made, local., local industries, local manufactur, local any, local carpenters, locals, local ltd.]"
1,"[fugo, fug, fugo company, fuo, fugo feeds, fugo ltd, fungo, fugo vits, fugo unga, fugo feed, fugo vit]"
2,[mukurwe-ini dairy]
3,"[xxxx, xxx]"
4,[g&g]
...,...
9457,[eastern flour mills]
9458,[kiresi seed ltd]
9459,[triachem tanzania ltd]
9460,[tags limted]


In [16]:
df_unique_match = df_non_dup['match'].drop_duplicates(keep='first').to_frame().reset_index(drop=True)
df_unique_match = pd.concat([df_unique_match, df_similar_strings],axis = 1)
df_unique_match.head()

,match,similar_strings
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","[local, lcal, loal, locl, local,, localy, local manufacture, local supplier, local nairobi, local manufacturer, local artisan, local juakali, loca, locali, local providers, local seller, localy made, local., local industries, local manufactur, local any, local carpenters, locals, local ltd.]"
1,"[fugo, fungo, fuo, fug]","[fugo, fug, fugo company, fuo, fugo feeds, fugo ltd, fungo, fugo vits, fugo unga, fugo feed, fugo vit]"
2,[mukurwe-ini],[mukurwe-ini dairy]
3,"[xxxx, xxx]","[xxxx, xxx]"
4,[g&g],[g&g]


In [17]:
dnd_df_unique_match = df_unique_match.copy()

In [18]:
dnd_df_unique_match

,match,similar_strings
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","[local, lcal, loal, locl, local,, localy, local manufacture, local supplier, local nairobi, local manufacturer, local artisan, local juakali, loca, locali, local providers, local seller, localy made, local., local industries, local manufactur, local any, local carpenters, locals, local ltd.]"
1,"[fugo, fungo, fuo, fug]","[fugo, fug, fugo company, fuo, fugo feeds, fugo ltd, fungo, fugo vits, fugo unga, fugo feed, fugo vit]"
2,[mukurwe-ini],[mukurwe-ini dairy]
3,"[xxxx, xxx]","[xxxx, xxx]"
4,[g&g],[g&g]
...,...,...
9457,[eastern flour],[eastern flour mills]
9458,[kiresi seed],[kiresi seed ltd]
9459,[triachem tanzania],[triachem tanzania ltd]
9460,[tags],[tags limted]


In [19]:
df_unique_match = dnd_df_unique_match
df_unique_match['average_length'] = df_unique_match['similar_strings'].apply(lambda x: round(sum(len(word.split()) for word in x) / len(x), 0))
df_unique_match

,match,similar_strings,average_length
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","[local, lcal, loal, locl, local,, localy, local manufacture, local supplier, local nairobi, local manufacturer, local artisan, local juakali, loca, locali, local providers, local seller, localy made, local., local industries, local manufactur, local any, local carpenters, locals, local ltd.]",2.0
1,"[fugo, fungo, fuo, fug]","[fugo, fug, fugo company, fuo, fugo feeds, fugo ltd, fungo, fugo vits, fugo unga, fugo feed, fugo vit]",2.0
2,[mukurwe-ini],[mukurwe-ini dairy],2.0
3,"[xxxx, xxx]","[xxxx, xxx]",1.0
4,[g&g],[g&g],1.0
...,...,...,...
9457,[eastern flour],[eastern flour mills],3.0
9458,[kiresi seed],[kiresi seed ltd],3.0
9459,[triachem tanzania],[triachem tanzania ltd],3.0
9460,[tags],[tags limted],2.0


In [20]:
df_unique_match['similar_strings'] = df_unique_match['similar_strings'].apply(lambda x: ' '.join(x))
df_unique_match

,match,similar_strings,average_length
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.",2.0
1,"[fugo, fungo, fuo, fug]",fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,2.0
2,[mukurwe-ini],mukurwe-ini dairy,2.0
3,"[xxxx, xxx]",xxxx xxx,1.0
4,[g&g],g&g,1.0
...,...,...,...
9457,[eastern flour],eastern flour mills,3.0
9458,[kiresi seed],kiresi seed ltd,3.0
9459,[triachem tanzania],triachem tanzania ltd,3.0
9460,[tags],tags limted,2.0


In [21]:
df_unique_match[:50]

,match,similar_strings,average_length
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.",2.0
1,"[fugo, fungo, fuo, fug]",fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,2.0
2,[mukurwe-ini],mukurwe-ini dairy,2.0
3,"[xxxx, xxx]",xxxx xxx,1.0
4,[g&g],g&g,1.0
5,"[rainbow, rainbows, rain bow, raibow, rianbow, rainvow, rainboe, rainbaw]",rainbow rainbow agrochemicals rainbow agro rainbow agri rainbow pharm rain bow agro rainbow biosciences raibow rainvow rianbow rainbow ltd rainboe rainbow chemical rainbow chemicals rainbow agricare rainbow science rainbaw rainbows rain bow agro s,2.0
6,"[ttt, tttt]",ttt tttt,1.0
7,[osho],osho osho ferrox osho industry osho ltd osho limited osho company osho chemical osho chemicals osho chemical.,2.0
8,[chloride],chloride exide chloride oxide chloride ltd chloride,2.0
9,"[bovtans general, bovitans general, bovtans geeral, bovtan general, bovitan general]",bovtans general merchants bovtan general merchant bovtan general mechart bovitans general marchants bovtans general merchant bovitans general merchants bovitan general merchant bovtans general minerals bovitan general merchants bovtans geeral merchants bovtans general mechants,3.0


In [63]:
# extracting most common words from each cluster in order
cluster_word_freq = {}


for id, row in df_unique_match.iterrows():
    cluster = row['similar_strings']
    
    words = re.split(r'\s+|-|\(|\)|/|\\|\||,', cluster)
    for word in words:
        if id in cluster_word_freq:
            cluster_word_freq[id][word] = cluster_word_freq[id].get(word, 0) + 1
        else:
            cluster_word_freq[id] = {word: 1}
    
for id in cluster_word_freq:
    cluster_word_freq[id] = sorted(cluster_word_freq[id].items(), key=lambda x: x[1], reverse=True)
    # cluster_word_freq[id] = list(cluster_word_freq[id].items())
    # cluster_word_freq[id] = cluster_word_freq[id].items()

cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
cluster_word_freq_df.rename(columns={0: 'id', 1: 'word_freq'}, inplace=True)
cluster_word_freq_df.head()

,id,word_freq
0,0,"[(l, 52), (a, 37), (c, 27), (o, 25), (r, 13), (e, 10), (u, 9), (i, 9), (n, 8), (s, 8), (t, 7), (m, 4), (p, 4), (d, 4), (y, 3), (f, 3), (, 2), (., 2), (b, 1), (j, 1), (k, 1), (v, 1)]"
1,1,"[(f, 13), (u, 12), (g, 11), (o, 11), (e, 4), (n, 3), (d, 3), (t, 3), (a, 2), (s, 2), (v, 2), (i, 2), (c, 1), (m, 1), (p, 1), (y, 1), (l, 1)]"
2,2,"[(i, 3), (u, 2), (r, 2), (, 2), (m, 1), (k, 1), (w, 1), (e, 1), (n, 1), (d, 1), (a, 1), (y, 1)]"
3,3,"[(x, 7)]"
4,4,"[(g, 2), (&, 1)]"


In [45]:
# # extracting most common words from each cluster in order
# cluster_word_freq = {}


# for id, row in df_unique_match.iterrows():
#     cluster = row['similar_strings']
    
#     words = re.split(r'\s+|-|\(|\)|/|\\|\||,', cluster)
#     for word in words:
#         if id in cluster_word_freq:
#             cluster_word_freq[id][word] = cluster_word_freq[id].get(word, 0) + 1
#         else:
#             cluster_word_freq[id] = {word: 1}
    
# for id in cluster_word_freq:
#     # cluster_word_freq[id] = sorted(cluster_word_freq[id].items(), key=lambda x: x[1], reverse=True)
#     cluster_word_freq[id] = list(cluster_word_freq[id].items())
#     # cluster_word_freq[id] = cluster_word_freq[id].items()

# cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
# cluster_word_freq_df.rename(columns={0: 'id', 1: 'word_freq'}, inplace=True)
# cluster_word_freq_df.head()

In [47]:
cluster_word_freq_df = pd.concat([cluster_word_freq_df, df_unique_match[['match', 'average_length']]], axis=1)
cluster_word_freq_df

,id,word_freq,match,average_length
0,0,"[(local, 15), (localy, 2), (lcal, 1), (loal, 1), (locl, 1), (, 1), (manufacture, 1), (supplier, 1), (nairobi, 1), (manufacturer, 1), (artisan, 1), (juakali, 1), (loca, 1), (locali, 1), (providers, 1), (seller, 1), (made, 1), (local., 1), (industries, 1), (manufactur, 1), (any, 1), (carpenters, 1), (locals, 1), (ltd., 1)]","[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]",2.0
1,1,"[(fugo, 8), (fug, 1), (company, 1), (fuo, 1), (feeds, 1), (ltd, 1), (fungo, 1), (vits, 1), (unga, 1), (feed, 1), (vit, 1)]","[fugo, fungo, fuo, fug]",2.0
2,2,"[(mukurwe, 1), (ini, 1), (dairy, 1)]",[mukurwe-ini],2.0
3,3,"[(xxxx, 1), (xxx, 1)]","[xxxx, xxx]",1.0
4,4,"[(g&g, 1)]",[g&g],1.0
...,...,...,...,...
9457,9457,"[(eastern, 1), (flour, 1), (mills, 1)]",[eastern flour],3.0
9458,9458,"[(kiresi, 1), (seed, 1), (ltd, 1)]",[kiresi seed],3.0
9459,9459,"[(triachem, 1), (tanzania, 1), (ltd, 1)]",[triachem tanzania],3.0
9460,9460,"[(tags, 1), (limted, 1)]",[tags],2.0


In [ ]:
dnd_cluster_word_freq_df = cluster_word_freq_df.copy()

In [55]:
# cluster_word_freq_df['cluster_name'] = cluster_word_freq_df['word_freq'].apply(lambda x: ' '.join(word[0] for word in x[:3]))
# cluster_word_freq_df.head()


for i, row in cluster_word_freq_df.iterrows():
    lst = row['word_freq']
    number = int(row['average_length'])
    # print(number)
    cluster_name = ' '.join(word[0] for word in lst[:number])
    cluster_word_freq_df.at[i, 'cluster_name'] = cluster_name
    
cluster_word_freq_df.head()

,id,word_freq,match,average_length,cluster_name
0,0,"[(local, 15), (localy, 2), (lcal, 1), (loal, 1), (locl, 1), (, 1), (manufacture, 1), (supplier, 1), (nairobi, 1), (manufacturer, 1), (artisan, 1), (juakali, 1), (loca, 1), (locali, 1), (providers, 1), (seller, 1), (made, 1), (local., 1), (industries, 1), (manufactur, 1), (any, 1), (carpenters, 1), (locals, 1), (ltd., 1)]","[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]",2.0,local localy
1,1,"[(fugo, 8), (fug, 1), (company, 1), (fuo, 1), (feeds, 1), (ltd, 1), (fungo, 1), (vits, 1), (unga, 1), (feed, 1), (vit, 1)]","[fugo, fungo, fuo, fug]",2.0,fugo fug
2,2,"[(mukurwe, 1), (ini, 1), (dairy, 1)]",[mukurwe-ini],2.0,mukurwe ini
3,3,"[(xxxx, 1), (xxx, 1)]","[xxxx, xxx]",1.0,xxxx
4,4,"[(g&g, 1)]",[g&g],1.0,g&g


In [23]:
# cluster_word_freq_df.to_csv('cluster_word_freq_df.csv', index=False)
cluster_word_freq_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v2/cluster_word_freq_df.csv')
cluster_word_freq_df.head()

,id,word_freq,match,average_length,cluster_name
0,0,"[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]","['local', 'localy', 'locals', 'locali', 'local.', 'local,', 'locl', 'loca', 'loal', 'lcal']",2.0,local localy
1,1,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]","['fugo', 'fungo', 'fuo', 'fug']",2.0,fugo fug
2,2,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",['mukurwe-ini'],2.0,mukurwe ini
3,3,"[('xxxx', 1), ('xxx', 1)]","['xxxx', 'xxx']",1.0,xxxx
4,4,"[('g&g', 1)]",['g&g'],1.0,g&g


In [24]:
cluster_word_freq_df[50:100]

,id,word_freq,match,average_length,cluster_name
50,50,"[('led', 2), ('bulb', 1)]",['led'],2.0,led bulb
51,51,"[('sunlight', 18), ('herb', 17), ('medicament', 10), ('baoding', 9), ('boading', 8), ('ltd', 7), ('co', 6), ('co.', 2), ('co.ltd', 2), ('boarding', 2), ('ltd.', 2), ('sunliht', 1), ('herd', 1), ('medication', 1), ('medlcament', 1), ('medcament', 1), ('mgt', 1)]","['baoding sunlight', 'boading sunlight', 'boarding sunlight', 'boading sunliht', 'baoding sunlight herb']",5.0,sunlight herb medicament baoding boading
52,52,"[('ordinary', 1), ('ordiary', 1)]","['ordinary', 'ordiary']",1.0,ordinary
53,53,"[('agribase', 4), ('bioscience', 2), ('agri', 1), ('base', 1), ('agrbase', 1), ('ltd', 1)]","['agribase', 'agri-base', 'agrbase']",2.0,agribase bioscience
54,54,"[('exceed', 3), ('24', 1), ('seed', 1), ('exceed24', 1), ('limited', 1)]","['exceed', 'exceed24']",2.0,exceed 24
55,55,"[('yhhy', 1), ('hhy', 1)]","['yhhy', 'hhy']",1.0,yhhy
56,56,"[('mastermind', 1), ('electronics', 1)]",['mastermind'],2.0,mastermind electronics
57,57,"[('montajat', 1), ('muntajat', 1), ('ltd', 1)]","['montajat', 'muntajat']",2.0,montajat muntajat
58,58,"[('rithophos', 1)]",['rithophos'],1.0,rithophos
59,59,"[('1', 1)]",['1'],1.0,1


In [25]:
# loading the master list
master_list_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v1/master_list.csv')
master_list_df

,combined_name,product_name,manufacturer_name
0,aviboost aqua block; vital animal health,aviboost aqua block,vital animal health
1,aviboost cl-x blue; vital animal health,aviboost cl-x blue,vital animal health
2,aviboost nutri block; vital animal health,aviboost nutri block,vital animal health
3,aviboost spectrum; vital animal health,aviboost spectrum,vital animal health
4,aviboost poultry tonic; vital animal health,aviboost poultry tonic,vital animal health
...,...,...,...
8774,jayhawk 500sc suspension concentrate ;,jayhawk 500sc suspension concentrate,NaN
8775,cynara 72 wp wettable powder;,cynara 72 wp wettable powder,NaN
8776,twiga-amine 720sl soluble concentrate;,twiga-amine 720sl soluble concentrate,NaN
8777,ranman top 160 sc suspension concentrate;,ranman top 160 sc suspension concentrate,NaN


In [26]:
cluster_word_freq_df['cluster_name'] = cluster_word_freq_df['cluster_name'].astype('str')
master_list_df['manufacturer_name'] = master_list_df['manufacturer_name'].astype('str')

In [26]:
matches_cache = {}
master_list = master_list_df['manufacturer_name'].to_list()

def get_closest_match(word, possibilities: list[str]):
    word = str(word).lower()
    if found := matches_cache.get(word):
        return found

    matches = get_close_matches(word, possibilities, n=1, cutoff=0.0)
    match = matches[0] if matches else ''
    score = round(SequenceMatcher(None, word, match).ratio(), 2)
    found = {'best_match': match, 'best_score': score}
    matches_cache[word] = found

    return found         # pd.Series([word, match, score], index = ['product_name', 'match', 'score'])

found_df = cluster_word_freq_df['cluster_name'].apply(lambda x: get_closest_match(x, master_list))

In [27]:
len(matches_cache)

9347

In [28]:
len(cluster_word_freq_df)

9462

In [29]:
dnt_found_df = found_df.copy()

In [30]:
found_df = found_df.apply(pd.Series)
found_df.head()

,best_match,best_score
0,chrysal africa ltd.,0.45
1,nutreco africa,0.36
2,iprocure limited,0.44
3,apex,0.25
4,hebei yuangzheng,0.21


In [27]:
# found_df.to_csv('found_df.csv', index=False)
found_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/models/model_v2/found_df.csv')
found_df.head()

,best_match,best_score
0,chrysal africa ltd.,0.45
1,nutreco africa,0.36
2,iprocure limited,0.44
3,apex,0.25
4,hebei yuangzheng,0.21


In [28]:
cluster_word_freq_df = pd.concat([cluster_word_freq_df, found_df], axis=1)
cluster_word_freq_df.head()

,id,word_freq,match,average_length,cluster_name,best_match,best_score
0,0,"[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]","['local', 'localy', 'locals', 'locali', 'local.', 'local,', 'locl', 'loca', 'loal', 'lcal']",2.0,local localy,chrysal africa ltd.,0.45
1,1,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]","['fugo', 'fungo', 'fuo', 'fug']",2.0,fugo fug,nutreco africa,0.36
2,2,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",['mukurwe-ini'],2.0,mukurwe ini,iprocure limited,0.44
3,3,"[('xxxx', 1), ('xxx', 1)]","['xxxx', 'xxx']",1.0,xxxx,apex,0.25
4,4,"[('g&g', 1)]",['g&g'],1.0,g&g,hebei yuangzheng,0.21


In [29]:
cluster_word_freq_df[cluster_word_freq_df['best_score'] >= 0.82]

,id,word_freq,match,average_length,cluster_name,best_match,best_score
308,308,"[('', 2)]",['-'],1.0,nan,NaN,1.00
1914,1914,"[('ebl', 1)]","['ebl', 'kebl']",1.0,ebl,eabl,0.86
1930,1930,"[('eab', 1), ('esab', 1)]","['eab', 'esab', 'eabc']",1.0,eab,eabl,0.86
1953,1953,"[('bidco', 1), ('land', 1), (""o'lakes"", 1), ('unga', 1), ('feeds', 1)]",['bidco land'],4.0,bidco land o'lakes unga,bidco land o'lakes,0.88
2035,2035,"[('ltd', 2), ('wellstock', 2), ('well', 1), ('stock', 1), ('limited', 1)]","['well stock', 'wellstock']",2.0,ltd wellstock,wellstock,0.82
2288,2288,"[('nano', 1), ('fertiliser', 1), ('nan', 1), ('ngano', 1)]","['nano', 'ngano', 'nan']",1.0,nano,NaN,0.86
2677,2677,"[('farmbase', 4), ('ltd', 2), ('limited', 2), ('farm', 1), ('base', 1), ('tz', 1), ('', 1), ('farmmbase', 1), ('lmd', 1)]","['farm base', 'farmbase', 'farmmbase']",2.0,farmbase ltd,farmate ltd.,0.83
3439,3439,"[('bamburi', 1), ('', 1), ('rai', 1), ('simbi', 1)]",['bamburi'],2.0,bamburi,bamburi,0.93
5455,5455,"[('rentokill', 1)]",['rentokill'],1.0,rentokill,rentokil ltd.,0.82
5468,5468,"[('oak', 1), ('medical', 1), ('limited', 1)]","['oak medical', 'oakmedica']",3.0,oak medical limited,oak medica ltd.,0.82


In [30]:
cluster_word_freq_df['cluster_name'] = np.where(cluster_word_freq_df['best_score'] >= 0.83, cluster_word_freq_df['best_match'], cluster_word_freq_df['cluster_name'])

In [32]:
match = cluster_word_freq_df['match']
cluster_word_freq_df = cluster_word_freq_df.drop('match', axis=1)
cluster_word_freq_df.head()

,id,word_freq,average_length,cluster_name,best_match,best_score
0,0,"[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]",2.0,local localy,chrysal africa ltd.,0.45
1,1,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]",2.0,fugo fug,nutreco africa,0.36
2,2,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",2.0,mukurwe ini,iprocure limited,0.44
3,3,"[('xxxx', 1), ('xxx', 1)]",1.0,xxxx,apex,0.25
4,4,"[('g&g', 1)]",1.0,g&g,hebei yuangzheng,0.21


In [34]:
# df_unique_match = pd.concat([df_unique_match, cluster_word_freq_df], axis=1)
df_unique_match.drop(['id', 'best_match', 'best_score', 'average_length'], axis=1, inplace=True)
# df_unique_match.drop(['word_freq', 'match', 'average_length', 'cluster_name'], axis=1, inplace=True)
df_unique_match.head()

,match,similar_strings,word_freq,cluster_name
0,"[local, localy, locals, locali, local., local,, locl, loca, loal, lcal]","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.","[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]",local localy
1,"[fugo, fungo, fuo, fug]",fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]",fugo fug
2,[mukurwe-ini],mukurwe-ini dairy,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",mukurwe ini
3,"[xxxx, xxx]",xxxx xxx,"[('xxxx', 1), ('xxx', 1)]",xxxx
4,[g&g],g&g,"[('g&g', 1)]",g&g


In [35]:
len(df_non_dup)

18619

In [36]:
dnddd_df_unique_match = df_unique_match.copy()
dnddd_df_non_dup = df_non_dup.copy()

In [53]:
df_unique_match = dnddd_df_unique_match.copy()
df_non_dup = dnddd_df_non_dup.copy()

In [54]:
df_unique_match.columns

Index(['match', 'similar_strings', 'word_freq', 'cluster_name'], dtype='object')

In [55]:
df_unique_match['match'] = df_unique_match['match'].apply(lambda x: ' '.join(x))
df_non_dup['match'] = df_non_dup['match'].apply(lambda x: ' '.join(x))

In [56]:
df_non_dup = df_non_dup.merge(df_unique_match, how='left', on='match')
df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match,similar_strings,word_freq,cluster_name
0,local,oic ltd,0.15,1,local,"local localy locals locali local. local, locl loca loal lcal","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.","[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]",local localy
1,fugo,pz cussons,0.27,1,fugo,fugo fungo fuo fug,fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]",fugo fug
2,mukurwe-ini dairy,zagro-india,0.41,2,mukurwe-ini,mukurwe-ini,mukurwe-ini dairy,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",mukurwe ini
3,xxxx,apex,0.22,1,xxxx,xxxx xxx,xxxx xxx,"[('xxxx', 1), ('xxx', 1)]",xxxx
4,g&g,hebei yuangzheng,0.20,1,g&g,g&g,g&g,"[('g&g', 1)]",g&g


In [57]:
# cleanup function
def compare(row):
    comparison = {}
    i = row['manufacturer_name']
    prods_list = row[['cluster_name', 'best_manufacturer_match']].tolist()
    if isinstance(i, str):
       comparison.update({i: get_close_matches(i, prods_list, n=1, cutoff=0.1)})
    product_name = list(comparison.keys()) if comparison else None
    match = []
    score = []
    if comparison:
       for key, value in comparison.items():
           if value:
              match.append(value[0])
              score.append(round(SequenceMatcher(None, i, value[0]).ratio(), 2))
           else:
              match.append(None)
              score.append(None)
    else:
       match.append(None)
       score.append(None)
              
    return pd.Series([match, score], index = ['match', 'score'])

In [58]:
def convert_to_string(value):
    return str(value)

columns_to_convert = ['manufacturer_name', 'cluster_name', 'best_manufacturer_match']
df_non_dup[columns_to_convert] = df_non_dup[columns_to_convert].applymap(convert_to_string)

In [59]:
for i in tqdm(range(10), desc="Processing", ncols=80, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}"):
    df_non_dup[['final_match', 'score']] = [compare(row) for _, row in df_non_dup.iterrows()]
    df_non_dup.head()

Processing: 100%|████████████████████████████████████████████████████████| 10/10


In [60]:
df_non_dup['final_match'] = df_non_dup['final_match'].apply(lambda x: ' '.join(x))
df_non_dup['score'] = df_non_dup['score'].apply(lambda x: x[0])

df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match,similar_strings,word_freq,cluster_name,final_match,score
0,local,oic ltd,0.15,1,local,"local localy locals locali local. local, locl loca loal lcal","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.","[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]",local localy,local localy,0.59
1,fugo,pz cussons,0.27,1,fugo,fugo fungo fuo fug,fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]",fugo fug,fugo fug,0.67
2,mukurwe-ini dairy,zagro-india,0.41,2,mukurwe-ini,mukurwe-ini,mukurwe-ini dairy,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",mukurwe ini,mukurwe ini,0.71
3,xxxx,apex,0.22,1,xxxx,xxxx xxx,xxxx xxx,"[('xxxx', 1), ('xxx', 1)]",xxxx,xxxx,1.00
4,g&g,hebei yuangzheng,0.20,1,g&g,g&g,g&g,"[('g&g', 1)]",g&g,g&g,1.00


In [61]:
df_non_dup['go_to_match'] = np.where(df_non_dup['score'] >= 0.79, df_non_dup['final_match'], df_non_dup['cluster_name'])
df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match,similar_strings,word_freq,cluster_name,final_match,score,go_to_match
0,local,oic ltd,0.15,1,local,"local localy locals locali local. local, locl loca loal lcal","local lcal loal locl local, localy local manufacture local supplier local nairobi local manufacturer local artisan local juakali loca locali local providers local seller localy made local. local industries local manufactur local any local carpenters locals local ltd.","[('local', 15), ('localy', 2), ('lcal', 1), ('loal', 1), ('locl', 1), ('', 1), ('manufacture', 1), ('supplier', 1), ('nairobi', 1), ('manufacturer', 1), ('artisan', 1), ('juakali', 1), ('loca', 1), ('locali', 1), ('providers', 1), ('seller', 1), ('made', 1), ('local.', 1), ('industries', 1), ('manufactur', 1), ('any', 1), ('carpenters', 1), ('locals', 1), ('ltd.', 1)]",local localy,local localy,0.59,local localy
1,fugo,pz cussons,0.27,1,fugo,fugo fungo fuo fug,fugo fug fugo company fuo fugo feeds fugo ltd fungo fugo vits fugo unga fugo feed fugo vit,"[('fugo', 8), ('fug', 1), ('company', 1), ('fuo', 1), ('feeds', 1), ('ltd', 1), ('fungo', 1), ('vits', 1), ('unga', 1), ('feed', 1), ('vit', 1)]",fugo fug,fugo fug,0.67,fugo fug
2,mukurwe-ini dairy,zagro-india,0.41,2,mukurwe-ini,mukurwe-ini,mukurwe-ini dairy,"[('mukurwe', 1), ('ini', 1), ('dairy', 1)]",mukurwe ini,mukurwe ini,0.71,mukurwe ini
3,xxxx,apex,0.22,1,xxxx,xxxx xxx,xxxx xxx,"[('xxxx', 1), ('xxx', 1)]",xxxx,xxxx,1.00,xxxx
4,g&g,hebei yuangzheng,0.20,1,g&g,g&g,g&g,"[('g&g', 1)]",g&g,g&g,1.00,g&g


In [62]:
df_manufacturer

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
5,local,oic ltd,0.15
16,fugo,pz cussons,0.27
22,local,oic ltd,0.15
31,mukurwe-ini dairy,zagro-india,0.41
33,xxxx,apex,0.22
...,...,...,...
25782,jojemi agri ventures ltd.,afri ventures,0.67
25783,triachem tanzania ltd,agrichem africa ltd.,0.67
25784,tags limted,chiapas limited,0.67
25785,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67


In [63]:
df_manufacturer = pd.merge(df_manufacturer, df_non_dup[['manufacturer_name', 'go_to_match']], how='left',on='manufacturer_name')
df_manufacturer.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,go_to_match
0,local,oic ltd,0.15,local localy
1,fugo,pz cussons,0.27,fugo fug
2,local,oic ltd,0.15,local localy
3,mukurwe-ini dairy,zagro-india,0.41,mukurwe ini
4,xxxx,apex,0.22,xxxx


In [64]:
df_manufacturer[:50]

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,go_to_match
0,local,oic ltd,0.15,local localy
1,fugo,pz cussons,0.27,fugo fug
2,local,oic ltd,0.15,local localy
3,mukurwe-ini dairy,zagro-india,0.41,mukurwe ini
4,xxxx,apex,0.22,xxxx
5,g&g,hebei yuangzheng,0.20,g&g
6,rainbow,rainbow agro sciences ltd.,0.41,rainbow agro
7,ttt,aesthetics ltd.,0.32,ttt
8,osho,basco,0.20,osho ferrox
9,chloride exide,chemline africa ltd.,0.34,chloride exide


In [66]:
df_man = df_man.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
df_man

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,highchem essentials ltd.,highchem essentials ltd.,0.98
1,bimeda ltd.,bimeda ltd.,0.96
2,simlaw seeds company ltd.,simlaw seeds company ltd.,0.98
3,ultravetis east africa ltd.,ultravetis east africa ltd.,0.98
4,cooper k-brands ltd.,cooper k-brands ltd.,0.98
...,...,...,...
25782,jojemi agri ventures ltd.,afri ventures,0.67
25783,triachem tanzania ltd,agrichem africa ltd.,0.67
25784,tags limted,chiapas limited,0.67
25785,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67


In [67]:
dnd_df_man = df_man.copy()

In [68]:
df_man = pd.merge(df_man, df_manufacturer[['manufacturer_name', 'go_to_match']], how='left',on='manufacturer_name')
df_man

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,go_to_match
0,highchem essentials ltd.,highchem essentials ltd.,0.98,NaN
1,bimeda ltd.,bimeda ltd.,0.96,NaN
2,simlaw seeds company ltd.,simlaw seeds company ltd.,0.98,NaN
3,ultravetis east africa ltd.,ultravetis east africa ltd.,0.98,NaN
4,cooper k-brands ltd.,cooper k-brands ltd.,0.98,NaN
...,...,...,...,...
40092,tags limted,chiapas limited,0.67,tags limted
40093,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real
40094,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real
40095,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real


In [69]:
df_man['go_to_match'] = np.where(df_man['go_to_match'].isna(), df_man['best_manufacturer_match'], df_man['go_to_match'])
df_man

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,go_to_match
0,highchem essentials ltd.,highchem essentials ltd.,0.98,highchem essentials ltd.
1,bimeda ltd.,bimeda ltd.,0.96,bimeda ltd.
2,simlaw seeds company ltd.,simlaw seeds company ltd.,0.98,simlaw seeds company ltd.
3,ultravetis east africa ltd.,ultravetis east africa ltd.,0.98,ultravetis east africa ltd.
4,cooper k-brands ltd.,cooper k-brands ltd.,0.98,cooper k-brands ltd.
...,...,...,...,...
40092,tags limted,chiapas limited,0.67,tags limted
40093,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real
40094,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real
40095,regal pharmaceutical,hebei yuanzheng pharmaceutical,0.67,regal real


In [71]:
df_man = df_man[['manufacturer_name', 'go_to_match']]
df_man = df_man.drop_duplicates(subset=['manufacturer_name'], keep='first')
len(df_man)

20051

In [72]:
df_man.to_csv('final_clustered_manufacturers.csv', index = False)

In [73]:
df_man.isna().sum()

manufacturer_name    1
go_to_match          1
dtype: int64